<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-loiDeKleiber/Premier_reseau_neurones-Kleiber.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter)
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Mon premier réseau de neurones avec Keras

## Approximer une étrange loi de la nature directement à partir de données

Vous allez créer votre premier réseau de neurones pour approximer une étrange loi de la nature directement à partir de données.

La loi de Kleiber, formulée par le biologiste Max Kleiber dans les années 1930, postule que la consommation d'énergie (le métabolisme) des animaux, y compris les humains, varie comme la puissance 3/4 de leur masse corporelle. Cette loi fonctionne des bactéries jusqu'aux baleines mais demeure pour le moment sans explication physique ou géométrique satisfaisante.

https://fr.wikipedia.org/wiki/Loi_de_Kleiber

On peut légitimenent se questionner sur l'utilité d'un tel exercice? En fait, ce qui est «intéressant» c'est de constater que le réseau de neurones va apprendre «seul» à partir des données à approximer cette étrange loi de la nature.

**Note**: Il n'est pas important de comprendre le détail du code informatique pour le moment. Ne vous inquiétez pas, des explications détaillées suivront bientôt.


# Acquisition des données...

### Source des données:
http://sites.science.oregonstate.edu/~schaferd/Sleuth/data-sets.html

Le **Fichier ex0826.csv** contenu dans l'archive **sleuth3csv.zip** a été renommé **LoiDeKleiber.csv** avec une entête en français

Ramsey, F., & Schafer, D. (2012). The statistical sleuth: a course in methods of data analysis. Cengage Learning.


In [1]:
# Création d'un répertoire pour les données
! mkdir DATA

In [2]:
# Téléchargement des données depuis un référentiel sur le site GitHub
! wget "https://github.com/ClaudeCoulombe/VIARENA/blob/master/DATA/LoiDeKleiber.csv?raw=True" -O DATA/LoiDeKleiber.csv

--2024-07-13 08:13:55--  https://github.com/ClaudeCoulombe/VIARENA/blob/master/DATA/LoiDeKleiber.csv?raw=True
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ClaudeCoulombe/VIARENA/raw/master/DATA/LoiDeKleiber.csv [following]
--2024-07-13 08:13:55--  https://github.com/ClaudeCoulombe/VIARENA/raw/master/DATA/LoiDeKleiber.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ClaudeCoulombe/VIARENA/master/DATA/LoiDeKleiber.csv [following]
--2024-07-13 08:13:55--  https://raw.githubusercontent.com/ClaudeCoulombe/VIARENA/master/DATA/LoiDeKleiber.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

# Quelques lignes de code...

## Importation des bibliothèques Python

In [3]:
# Importation des bibliothèques Python
import tensorflow as tf
import numpy as np
import pandas as pd
print("Bibliothèques importées")

Bibliothèques importées


## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [4]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 1

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")


Germe aléatoire fixé


## Lecture des données

In [5]:
# Lecture des données
loi_kleiber_data = pd.read_csv("DATA/LoiDeKleiber.csv")
print("Données lues")

Données lues


## Affichage des données

In [6]:
# Affichage de l'information de base sur le ju de données
loi_kleiber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NomCommun    95 non-null     object 
 1   Espece       95 non-null     object 
 2   Masse        95 non-null     float64
 3   Metabolisme  95 non-null     float64
 4   DureeDeVie   95 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.8+ KB


Le jeu de données comporte 95 exemples et 5 attributs: 'NomCommun', 'Espece', 'Masse', 'Metabolisme', 'DureeDeVie'[texte du lien](https://)

In [7]:
# Affichage d'un échantillon de 5 exemplaires choisis au hasard du jeu de données
loi_kleiber_data.sample(n=5,random_state=42)

# masse en kg
# métabolisme en kJ par jour

,NomCommun,Espece,Masse,Metabolisme,DureeDeVie
68,Chamois,Rupicapra rupicapra,40.00,3140.0,21.0
22,Scaly anteater,Manis javanica,4.22,529.0,11.0
72,Wolverine,Gulogulo,12.70,2820.0,15.0
73,Badger,Meles meles,11.10,1440.0,16.0
0,Echidna,Tachiglossus aculeatus,2.50,302.0,14.0


## Prétraitement des données

Extraction des attributs qui vont servir à la prédiction ou variables indépendantes.

Rappelez-vous que nous cherchons à prédire le métabolisme à partir de la masse donc un seul attribut prédictif, la masse

In [8]:
from sklearn.preprocessing import StandardScaler

# Extraction des attributs qui vont servir è la prédiction ou variables indépendantes
# Rappelez-vous que nous cherchons à prédire le métabolisme à partir de la masse
# donc un seul attribut prédictif, la masse
attribut_predictif = loi_kleiber_data['Masse'].values.reshape(-1, 1)
# Normalisation de l'attribut-prédictif pour faciliter le travail de l'algorithme
normalisateur_attribut_predictif = StandardScaler()
normalisateur_attribut_predictif.fit(attribut_predictif)
attribut_predictif = normalisateur_attribut_predictif.transform(attribut_predictif)
print("Attribut prédictif extrait et normalisé")

Attribut prédictif extrait et normalisé


Extraction de l'attribut que l'on cherche à prédire ou attribut-cible ou variable dépendante.

Rappelez-vous que nous cherchons à prédire le métabolismemdonc un seul attribut-cible, le métabolisme



In [9]:

# Extraction de l'attribut à prédire ou attribut-cible ou variable dépendante
# Rappelez-vous que nous cherchons à prédire le métabolisme
# donc un seul attribut-cible, le métabolisme
attribut_cible = loi_kleiber_data['Metabolisme'].values.reshape(-1, 1)
# Normalisation de l'attribut-cible pour faciliter le travail de l'algorithme
normalisateur_attribut_cible = StandardScaler()
normalisateur_attribut_cible.fit(attribut_cible)
attribut_cible = normalisateur_attribut_cible.transform(attribut_cible)
print("Attribut-cible extrait et normalisé")

Attribut-cible extrait et normalisé


## Construction d'un réseau de neurones de type perceptron à trois couches:
### couche d'entrée, couche cachée, couche de sortie

In [10]:
# Construction d'un réseau de neurones de typ percptron à trois couches: couche d'entrée, couche cachée, couche de sortie
reseau_de_neurones = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1]),
                                                 tf.keras.layers.Dense(units=10, activation='relu'),
                                                 tf.keras.layers.Dense(units=1)])
# Affichage de l'architecture du réseau
print("Architecture du réseau de neurones:")
reseau_de_neurones.summary()



Architecture du réseau de neurones:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
 dense_1 (Dense)             (None, 10)                20        
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 33 (132.00 Byte)
Trainable params: 33 (132.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Compilation du réseau de neurones
### optimiseur: Adam
### taux d'apprentissage: 0.001
### fonction d'erreur: erreur quadratique moyenne ('mean_squared_error')



In [11]:
# Compilation du réseau de neurones - optimiseur: Adam, taux d'apprentissage: 0.001, fonction d'erreur: erreur quadratique moyenne (mean_squared_error)
reseau_de_neurones.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                           loss='mean_squared_error')

print("Réseau de neurones compilé")


Réseau de neurones compilé


## Entraînement du réseau de neurones sur les données:
### attribut_predictif et attribut_cible
### pendant 500 itérations ou époques

In [12]:
# Entraînement du réseau sur les données: variable_explicative et variable_dependante, pendant 500 itérations ou époques
traces = reseau_de_neurones.fit(attribut_predictif,attribut_cible,epochs=500,verbose=1)
# Affichage de l'erreur à la fin de l'entraînement
print("Erreur à la fin:",traces.history['loss'][-1])

Epoch 1/500
3/3 [==============================] - 1s 7ms/step - loss: 1.3312
Epoch 2/500
3/3 [==============================] - 0s 4ms/step - loss: 1.3194
Epoch 3/500
3/3 [==============================] - 0s 4ms/step - loss: 1.3131
Epoch 4/500
3/3 [==============================] - 0s 4ms/step - loss: 1.2971
Epoch 5/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2887
Epoch 6/500
3/3 [==============================] - 0s 4ms/step - loss: 1.2751
Epoch 7/500
3/3 [==============================] - 0s 4ms/step - loss: 1.2689
Epoch 8/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2560
Epoch 9/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2460
Epoch 10/500
3/3 [==============================] - 0s 6ms/step - loss: 1.2343
Epoch 11/500
3/3 [==============================] - 0s 6ms/step - loss: 1.2252
Epoch 12/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2183
Epoch 13/500
3/3 [==============================] - 0s 5ms/st

### Test du réseau de neurones qui a été entraîné
Vous allez valider la fonction que le réseau neuronal a apprise avec des données de test qui ne faisaient pas partie du jeu de données d'entraînement.

In [13]:
# Être humain, Homo sapiens
animal = "être humain"
masse = 6.50E+01
# Normalisation des données d'entrée
masse_normalisee = normalisateur_attribut_predictif.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 7.56E+03
# Application du modèle en inférence ou prédiction de la variable dépendante et «dénormalisation» du résultat
prediction_metabolisme = normalisateur_attribut_cible.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0]
# Affichage de la prédiction, de la vraie valeur (mesurée) et de l'écart (ou erreur) en %
print("\nAnimal:", animal,
      ", Masse:", masse,
      ", prédiction du métabolisme:", round(prediction_metabolisme,2),
      ", Vraie valeur du métabolisme:", vraie_valeur_metabolisme,
      ", écart en %:", round((vraie_valeur_metabolisme-prediction_metabolisme)/vraie_valeur_metabolisme*100,2),"%")


1/1 [==============================] - 0s 118ms/step

Animal: être humain , Masse: 65.0 , prédiction du métabolisme: 8333.68 , Vraie valeur du métabolisme: 7560.0 , écart en %: -10.23 %


In [14]:
# Chat, Felis silvestris, 3.00E+00, 5.46E+02, 11
animal = "chat"
masse = 3.00E+00
# Normalisation des données d'entrée
masse_normalisee = normalisateur_attribut_predictif.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 5.46E+02
# Application du modèle en inférence ou prédiction de l'attribut-cible et «dénormalisation» du résultat
prediction_metabolisme = normalisateur_attribut_cible.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0]
# Affichage de la prédiction, de la vraie valeur (mesurée) et de l'écart (ou erreur) en %
print("\nAnimal:", animal,
      ", Masse:", masse,
      ", prédiction du métabolisme:", round(prediction_metabolisme,2),
      ", Vraie valeur du métabolisme:", vraie_valeur_metabolisme,
      ", écart en %:", round((vraie_valeur_metabolisme-prediction_metabolisme)/vraie_valeur_metabolisme*100,2),"%")



1/1 [==============================] - 0s 31ms/step

Animal: chat , Masse: 3.0 , prédiction du métabolisme: 507.45 , Vraie valeur du métabolisme: 546.0 , écart en %: 7.06 %


In [15]:
# Cheval, Equus cabalus, 4.00E+02, 3.20E+04, 40
animal = "cheval"
masse = 4.00E+02
# Normalisation des données d'entrée
masse_normalisee = normalisateur_attribut_predictif.transform(np.array([masse]).reshape(-1, 1))
vraie_valeur_metabolisme = 3.20E+04
# Application du modèle en inférence ou prédiction de la variable dépendante et «dénormalisation» du résultat
prediction_metabolisme = normalisateur_attribut_cible.inverse_transform(reseau_de_neurones.predict(masse_normalisee))[0][0]
# Affichage de la prédiction, de la vraie valeur (mesurée) et de l'écart (ou erreur) en %
print("\nAnimal:", animal,
      ", Masse:", masse,
      ", prédiction du métabolisme:", round(prediction_metabolisme,2),
      ", Vraie valeur du métabolisme:", vraie_valeur_metabolisme,
      ", écart en %:", round((vraie_valeur_metabolisme-prediction_metabolisme)/vraie_valeur_metabolisme*100,2),"%")


1/1 [==============================] - 0s 32ms/step

Animal: cheval , Masse: 400.0 , prédiction du métabolisme: 31611.03 , Vraie valeur du métabolisme: 32000.0 , écart en %: 1.22 %


### Conclusion

Vous pouvez constater que le réseau de neurones retourne des valeurs assez proches des vraies valeurs données par la loi de Kleiber. Il est important de comprendre que le réseau de neurones n'apprend pas une formule exacte mais bien qu'il approxime itérativement une fonction.  

Choses à retenir:

* Le réseau de neurones est capable d'apprendre à approximer une fonction directement à partir des données
* Le processus d'apprentissage est itératif
    

In [16]:
print("Exécution du carnet web IPython terminée")

Exécution du carnet web IPython terminée
